## 协作型过滤
通常的做法是对一大群人进行搜索，找出与我们品味相近的一群人

## 搜集偏好
我们用**嵌套的字典**表示不同人及其偏好的方法

In [2]:
critics={'Lisa Rose': {'Lady in the Water': 2.5, 'Snakes on a Plane': 3.5,
 'Just My Luck': 3.0, 'Superman Returns': 3.5, 'You, Me and Dupree': 2.5, 
 'The Night Listener': 3.0},
'Gene Seymour': {'Lady in the Water': 3.0, 'Snakes on a Plane': 3.5, 
 'Just My Luck': 1.5, 'Superman Returns': 5.0, 'The Night Listener': 3.0, 
 'You, Me and Dupree': 3.5}, 
'Michael Phillips': {'Lady in the Water': 2.5, 'Snakes on a Plane': 3.0,
 'Superman Returns': 3.5, 'The Night Listener': 4.0},
'Claudia Puig': {'Snakes on a Plane': 3.5, 'Just My Luck': 3.0,
 'The Night Listener': 4.5, 'Superman Returns': 4.0, 
 'You, Me and Dupree': 2.5},
'Mick LaSalle': {'Lady in the Water': 3.0, 'Snakes on a Plane': 4.0, 
 'Just My Luck': 2.0, 'Superman Returns': 3.0, 'The Night Listener': 3.0,
 'You, Me and Dupree': 2.0}, 
'Jack Matthews': {'Lady in the Water': 3.0, 'Snakes on a Plane': 4.0,
 'The Night Listener': 3.0, 'Superman Returns': 5.0, 'You, Me and Dupree': 3.5},
'Toby': {'Snakes on a Plane':4.5,'You, Me and Dupree':1.0,'Superman Returns':4.0}}

可以很容易的对字典进行查询和修改

In [3]:
critics['Lisa Rose']['Lady in the Water']

2.5

In [4]:
critics['Toby']

{'Snakes on a Plane': 4.5, 'Superman Returns': 4.0, 'You, Me and Dupree': 1.0}

## 寻找相近的用户
我们需要一种方法来确定人们在品味方面的相似度。

我们可以将每个人与其它人进行对比，计算他们的相似度评价值。

这可以通过很多种方式，这里介绍两种：
- 欧几里得距离
- 皮尔逊相关度

### 欧几里得距离
具体的解释可以参见[维基百科](https://www.wikiwand.com/zh-hans/%E6%AC%A7%E5%87%A0%E9%87%8C%E5%BE%97%E8%B7%9D%E7%A6%BB)

In [5]:
from math import sqrt

# 返回一个有关person1和person2的基于距离的相似度评价
def sim_distance(prefs,person1,person2):
    si={}
    for item in prefs[person1]: 
        if item in prefs[person2]: si[item]=1

    # 如果两者没有共同之处，返回0
    if len(si)==0: 
        return 0

    # 计算所有差值的平方和
    sum_of_squares=sum([pow(prefs[person1][item]-prefs[person2][item],2) for item in prefs[person1] if item in prefs[person2]])

    return 1/(1+sum_of_squares)

In [6]:
sim_distance(critics,'Lisa Rose','Gene Seymour')

0.14814814814814814

### 皮尔逊相关度
是判断数据与某一直线拟合程度的度量。在数据不是很规范（例如，影评者对影片的评价总是相对平均水平偏差很大），会倾向于给出更号的结果

![](https://sfault-image.b0.upaiyun.com/91/40/91408e2911e430ab46edbc49b343d7b9_articlex)

具体可以参加[维基百科](https://www.wikiwand.com/zh-hans/%E7%9A%AE%E5%B0%94%E9%80%8A%E7%A7%AF%E7%9F%A9%E7%9B%B8%E5%85%B3%E7%B3%BB%E6%95%B0)

In [7]:
def sim_pearson(prefs,p1,p2):
    # 得到双方都评价过的列表
    si={}
    for item in prefs[p1]: 
        if item in prefs[p2]: si[item]=1

    #  如果两者列表的没有共同之处，返回1
    if len(si)==0: 
        return 1

    # 得到列表相同之处
    n=len(si)
  
    # 对所有偏好求和
    sum1=sum([prefs[p1][it] for it in si])
    sum2=sum([prefs[p2][it] for it in si])
  
    # 求平方和
    sum1Sq=sum([pow(prefs[p1][it],2) for it in si])
    sum2Sq=sum([pow(prefs[p2][it],2) for it in si])	
  
    # 乘积之和
    pSum=sum([prefs[p1][it]*prefs[p2][it] for it in si])
  
    # 计算皮尔逊值
    num=pSum-(sum1*sum2/n)
    den=sqrt((sum1Sq-pow(sum1,2)/n)*(sum2Sq-pow(sum2,2)/n))
    if den==0: 
        return 0

    r=num/den

    return r

In [8]:
sim_pearson(critics,'Lisa Rose','Gene Seymour')

0.39605901719066977

## 为评论者打分
找出与指定人员最相近的n个人

In [9]:
def topMatches(prefs,person,n=5,similarity=sim_pearson):
  scores=[(similarity(prefs,person,other),other) 
                  for other in prefs if other!=person]
  scores.sort()
  scores.reverse()
  return scores[0:n]

In [10]:
topMatches(critics,'Lisa Rose',n=3)

[(0.9912407071619299, 'Toby'),
 (0.7470178808339965, 'Jack Matthews'),
 (0.5940885257860044, 'Mick LaSalle')]

## 推荐物品
一般情况下，我们不是要得到趣味相投的影评者，而是要得到一份影片的推荐。

我们通过一个经过加权的评价值来对影片打分。

In [15]:
def getRecommendations(prefs,person,similarity=sim_pearson):
    totals={}
    simSums={}
    for other in prefs:
    # 不和自己做比较
        if other==person: 
            continue
        sim=similarity(prefs,person,other)

    # 忽略评价值为零或者小与零的情况
        if sim<=0: 
            continue
        for item in prefs[other]:
            if item not in prefs[person] or prefs[person][item]==0:
                # 相似度乘以评分
                totals.setdefault(item , 0)
                totals[item]+=prefs[other][item]*sim
                # 相似度之和
                simSums.setdefault(item,0)
                simSums[item]+=sim

    rankings=[(total/simSums[item],item) for item,total in totals.items()]

    rankings.sort()
    rankings.reverse()
    return rankings

In [16]:
getRecommendations(critics,'Toby')

[(3.3477895267131013, 'The Night Listener'),
 (2.8325499182641614, 'Lady in the Water'),
 (2.5309807037655645, 'Just My Luck')]

## 匹配商品
我们能不能知道哪些商品是彼此接近的？

和之前人与人的相似度度量方法类似，只要将人员和物品相互调换就行了

In [17]:
def transformPrefs(prefs):
    result={}
    for person in prefs:
        for item in prefs[person]:
            result.setdefault(item,{})
      
             # 调换物品和人
            result[item][person]=prefs[person][item]
    return result


In [18]:
movies=transformPrefs(critics)

In [19]:
topMatches(movies,'Superman Returns')

[(0.6579516949597695, 'You, Me and Dupree'),
 (0.4879500364742689, 'Lady in the Water'),
 (0.11180339887498941, 'Snakes on a Plane'),
 (-0.1798471947990544, 'The Night Listener'),
 (-0.42289003161103106, 'Just My Luck')]

我们还可以通过影片推荐评论者

In [20]:
getRecommendations(movies,'Just My Luck')

[(4.0, 'Michael Phillips'), (3.0, 'Jack Matthews')]

## 基于物品的过滤
对于大型的网站，将一个用户和其它所有用户进行比较，速度太慢了。这是无法忍受的。

我们可以采用**基于物品的过滤**方式。

其具体的思想是：
- 为每件物品预先计算好最近的其它物品
- 如果想为用户A推荐时，找出其好评的电影，比如说10部。
- 找出这10部电影的相似电影，通过评分和相似度相乘的和，再处于相似度的和，这样来找出"最可能喜欢"的电影

它的不同之处在于：**物品间的比较不会像用户比较那么频繁**

## 构造物品比较数据集
首先编写一个函数，包含相似物品的完整数据集

In [22]:
def calculateSimilarItems(prefs,n=10):
    # 建立字典，给出这些物品所有相近的其他物品
    result={}

    itemPrefs=transformPrefs(prefs)
    c=0
    for item in itemPrefs:
    # 针对大数据量更新状态
        c+=1
        if c%100==0: 
            print("%d / %d" % (c,len(itemPrefs)))
        # 寻找最为相近的物品
        scores=topMatches(itemPrefs,item,n=n,similarity=sim_distance)
        result[item]=scores
    return result

In [25]:
itemsim=calculateSimilarItems(critics)

## 获得推荐


In [26]:
def getRecommendedItems(prefs,itemMatch,user):
    userRatings=prefs[user]
    scores={}
    totalSim={}
    # 循环遍历由当前用户评分的物品
    for (item,rating) in userRatings.items( ):

        # 循环遍历与当前物品相近的物品
        for (similarity,item2) in itemMatch[item]:

            if item2 in userRatings: continue
            # 评价值和相似度加权
            scores.setdefault(item2,0)
            scores[item2]+=similarity*rating
            # 全部相似度之和
            totalSim.setdefault(item2,0)
            totalSim[item2]+=similarity

    # 将每个合计值除以加权值，求出平均值
    rankings=[(score/totalSim[item],item) for item,score in scores.items( )]

    # 排序
    rankings.sort( )
    rankings.reverse( )
    return rankings

In [27]:
getRecommendedItems(critics,itemsim,'Toby')

[(3.1826347305389224, 'The Night Listener'),
 (2.5983318700614575, 'Just My Luck'),
 (2.4730878186968837, 'Lady in the Water')]

## 使用MovieLens数据集
这里使用明尼苏达大学的涉及电影评价的真实数据集

下载地址：[ml-100k.zip](http://files.grouplens.org/datasets/movielens/ml-100k.zip)

100,000 ratings from 1000 users on 1700 movies

In [29]:
def loadMovieLens(path='data'):
    # 获取影片标题
    movies={}
    for line in open(path+'/u.item'):
        (id,title)=line.split('|')[0:2]
        movies[id]=title
  
    # 加载数据
    prefs={}
    for line in open(path+'/u.data'):
        (user,movieid,rating,ts)=line.split('\t')
        prefs.setdefault(user,{})
        prefs[user][movies[movieid]]=float(rating)
    return prefs

In [31]:
prefs=loadMovieLens()

In [32]:
prefs['87']

{'2001: A Space Odyssey (1968)': 5.0,
 'Ace Ventura: Pet Detective (1994)': 4.0,
 'Addams Family Values (1993)': 2.0,
 'Addicted to Love (1997)': 4.0,
 'Adventures of Priscilla, Queen of the Desert, The (1994)': 3.0,
 'Adventures of Robin Hood, The (1938)': 5.0,
 'Air Force One (1997)': 3.0,
 'Air Up There, The (1994)': 3.0,
 'Alien (1979)': 4.0,
 'American President, The (1995)': 5.0,
 'Annie Hall (1977)': 4.0,
 'Apocalypse Now (1979)': 4.0,
 'Babe (1995)': 5.0,
 'Baby-Sitters Club, The (1995)': 2.0,
 'Back to the Future (1985)': 5.0,
 'Bad Boys (1995)': 4.0,
 'Bananas (1971)': 5.0,
 'Barcelona (1994)': 3.0,
 'Batman & Robin (1997)': 4.0,
 'Batman (1989)': 3.0,
 'Batman Returns (1992)': 3.0,
 'Big Green, The (1995)': 3.0,
 'Big Squeeze, The (1996)': 2.0,
 'Birdcage, The (1996)': 4.0,
 'Blade Runner (1982)': 4.0,
 'Blues Brothers, The (1980)': 5.0,
 'Boomerang (1992)': 3.0,
 'Boot, Das (1981)': 4.0,
 'Brady Bunch Movie, The (1995)': 2.0,
 'Braveheart (1995)': 4.0,
 'Bridge on the River

我们可以基于用户推荐

In [35]:
getRecommendations(prefs,'87')[0:30]

[(5.0, 'They Made Me a Criminal (1939)'),
 (5.0, 'Star Kid (1997)'),
 (5.0, 'Santa with Muscles (1996)'),
 (5.0, 'Saint of Fort Washington, The (1993)'),
 (5.0, 'Marlene Dietrich: Shadow and Light (1996) '),
 (5.0, 'Great Day in Harlem, A (1994)'),
 (5.0, 'Entertaining Angels: The Dorothy Day Story (1996)'),
 (5.0, 'Boys, Les (1997)'),
 (4.89884443128923, 'Legal Deceit (1997)'),
 (4.815019082242709, 'Letter From Death Row, A (1998)'),
 (4.800260666069043, 'Mrs. Dalloway (1997)'),
 (4.771240079753504, 'Leading Man, The (1996)'),
 (4.7321082983941425, 'Hearts and Minds (1996)'),
 (4.707354190896574, 'Dangerous Beauty (1998)'),
 (4.696244466490867, 'Pather Panchali (1955)'),
 (4.652397061026759, 'Lamerica (1994)'),
 (4.532337612572981, 'Innocents, The (1961)'),
 (4.5279985747470795, 'Casablanca (1942)'),
 (4.512903125553783, 'Four Days in September (1997)'),
 (4.510270149719864, 'Everest (1998)'),
 (4.485151301801342, 'Wallace & Gromit: The Best of Aardman Animation (1996)'),
 (4.46328746

改为基于物品的推荐

In [36]:
itemsim=calculateSimilarItems(prefs,n=50)

100 / 1664
200 / 1664
300 / 1664
400 / 1664
500 / 1664
600 / 1664
700 / 1664
800 / 1664
900 / 1664
1000 / 1664
1100 / 1664
1200 / 1664
1300 / 1664
1400 / 1664
1500 / 1664
1600 / 1664


In [37]:
getRecommendedItems(prefs,itemsim,'87')[0:30]

[(5.0, "What's Eating Gilbert Grape (1993)"),
 (5.0, 'Vertigo (1958)'),
 (5.0, 'Usual Suspects, The (1995)'),
 (5.0, 'Toy Story (1995)'),
 (5.0, 'Titanic (1997)'),
 (5.0, 'Sword in the Stone, The (1963)'),
 (5.0, 'Stand by Me (1986)'),
 (5.0, 'Sling Blade (1996)'),
 (5.0, 'Silence of the Lambs, The (1991)'),
 (5.0, 'Shining, The (1980)'),
 (5.0, 'Shine (1996)'),
 (5.0, 'Sense and Sensibility (1995)'),
 (5.0, 'Scream (1996)'),
 (5.0, 'Rumble in the Bronx (1995)'),
 (5.0, 'Rock, The (1996)'),
 (5.0, 'Robin Hood: Prince of Thieves (1991)'),
 (5.0, 'Reservoir Dogs (1992)'),
 (5.0, 'Police Story 4: Project S (Chao ji ji hua) (1993)'),
 (5.0, 'House of the Spirits, The (1993)'),
 (5.0, 'Fresh (1994)'),
 (5.0, 'Denise Calls Up (1995)'),
 (5.0, 'Day the Sun Turned Cold, The (Tianguo niezi) (1994)'),
 (5.0, 'Before the Rain (Pred dozhdot) (1994)'),
 (5.0, 'Assignment, The (1997)'),
 (5.0, '1-900 (1994)'),
 (4.875, "Ed's Next Move (1996)"),
 (4.833333333333333, 'Anna (1996)'),
 (4.8, 'Dark City 

运行过程中可以发现，时间主要耗费在构造物品的相似度字典上，推荐过程几乎是瞬间完成的